<a href="https://colab.research.google.com/github/MDaniyalTariq/Urdu_Handwritten_Words_Recognition_through_CNN_Ensembling/blob/main/lstm_dropout_bnn_l2_regularizationipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import json
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2

# Function to load and process the data
def load_data_from_directory(directory_path):
    data = []
    labels = []

    # Iterate over each file in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            file_path = os.path.join(directory_path, filename)

            # Load the data from the JSON file
            with open(file_path, "r") as f:
                file_data = json.load(f)

            # Extract sequences from the file
            sequences = []
            for character_variation in file_data:
                for i in range(0, len(character_variation), 5):  # Create sequences with 5 time steps
                    sequence = character_variation[i:i+5]
                    if len(sequence) == 5:
                        # Extract dx, dy, and timestamp from the dictionary and flatten into a list
                        flattened_sequence = [[point['dx'], point['dy'], point['timestamp']] for point in sequence]
                        sequences.append(flattened_sequence)

            # Add the sequences to the data list and label (filename without .json) to the labels list
            data.extend(sequences)
            labels.extend([filename.replace(".json", "")] * len(sequences))  # File name is the label

    return data, labels

# Preprocess the data (pad sequences and encode labels)
def preprocess_data(data, labels, sequence_length=50):
    # Pad or truncate sequences to make them of equal length
    sequences = pad_sequences(data, maxlen=sequence_length, dtype='float32', padding='post', truncating='post')

    # Encode the labels (character names)
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    labels_encoded = to_categorical(labels_encoded)  # One-hot encoding

    return np.array(sequences), np.array(labels_encoded), label_encoder

# Function to export the trained model
def save_model(model, model_path="trained_model.h5"):
    model.save(model_path)
    print(f"Model saved to {model_path}")

# Function to load the model
def load_trained_model(model_path="trained_model.h5"):
    return load_model(model_path)

# Main function to load, preprocess, and train the model
def main():
    # Load the data from your directory
    directory_path = "/content/dataset/json"
    data, labels = load_data_from_directory(directory_path)

    # Preprocess the data
    sequences, labels_encoded, label_encoder = preprocess_data(data, labels)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(sequences, labels_encoded, test_size=0.2, random_state=42)

    # Example model (LSTM with BatchNormalization, Dropout, and L2 regularization)
    model = Sequential([
        LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True,
             kernel_regularizer=l2(0.01)),  # Add L2 regularization
        BatchNormalization(),
        Dropout(0.3),  # Increase dropout to avoid overfitting
        LSTM(64, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(len(label_encoder.classes_), activation="softmax")
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Model checkpoint to save the best model

    model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

    # Train the model with early stopping and model checkpointing
    model.fit(X_train, y_train, epochs=5, batch_size=24, validation_data=(X_test, y_test),
              callbacks=[early_stopping, model_checkpoint])

    # Save the trained model
    save_model(model)



    # Save label encoder
    import joblib
    joblib.dump(label_encoder, "label_encoder.pkl")

# Function to predict a character from user input
def predict_character(user_input, model, label_encoder, sequence_length=50):
    """
    Predict the character based on the user input.
    :param user_input: List of dictionaries containing 'dx', 'dy', 'timestamp'.
    :param model: Trained model to use for predictions.
    :param label_encoder: Label encoder to decode predictions into characters.
    :param sequence_length: The sequence length for padding/truncating.
    :return: Predicted character label.
    """
    # Preprocess the user input
    flattened_input = [[point['dx'], point['dy'], point['timestamp']] for point in user_input]
    padded_input = pad_sequences([flattened_input], maxlen=sequence_length, dtype='float32', padding='post', truncating='post')

    # Make the prediction
    prediction = model.predict(padded_input)

    # Get the class index with the highest probability
    predicted_class_index = np.argmax(prediction)

    # Decode the class index to the actual character label
    predicted_character = label_encoder.inverse_transform([predicted_class_index])[0]

    return predicted_character

# Example of how to use the prediction function
if __name__ == "__main__":
    # Train and save the model first
    main()




/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                        │ (None, 50, 128)             │          67,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 50, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 50, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 35)                  │           2,275 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 120,035 (468.89 KB)

 Trainable params: 119,651 (467.39 KB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/5
3018/3018 ━━━━━━━━━━━━━━━━━━━━ 254s 82ms/step - accuracy: 0.0462 - loss: 4.0308 - val_accuracy: 0.0463 - val_loss: 3.5675
Epoch 2/5
3018/3018 ━━━━━━━━━━━━━━━━━━━━ 254s 84ms/step - accuracy: 0.0610 - loss: 3.3700 - val_accuracy: 0.0684 - val_loss: 3.3991
Epoch 3/5
3018/3018 ━━━━━━━━━━━━━━━━━━━━ 257s 83ms/step - accuracy: 0.0779 - loss: 3.2922 - val_accuracy: 0.0927 - val_loss: 3.2159
Epoch 4/5
3018/3018 ━━━━━━━━━━━━━━━━━━━━ 262s 83ms/step - accuracy: 0.0801 - loss: 3.2517 - val_accuracy: 0.0534 - val_loss: 3.5957
Epoch 5/5
3018/3018 ━━━━━━━━━━━━━━━━━━━━ 252s 83ms/step - accuracy: 0.0828 - loss: 3.2526 - val_accuracy: 0.0761 - val_loss: 3.3153


Model saved to trained_model.h5


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'best_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [10]:
import joblib
# Load the saved model and label encoder
model = load_trained_model("trained_model.h5")
label_encoder = joblib.load("label_encoder.pkl")

    # Sample user input (list of dictionaries with 'dx', 'dy', 'timestamp')
user_input = [
        {"dx": 198.5, "dy": 97, "timestamp": 0},
        {"dx": 198.5, "dy": 97, "timestamp": 1},
        {"dx": 198.5, "dy": 97, "timestamp": 26},
        {"dx": 198.5, "dy": 97, "timestamp": 26},
        {"dx": 198.5, "dy": 97, "timestamp": 50},
]

    # Predict the character
predicted_character = predict_character(user_input, model, label_encoder)

    # Print the predicted character
print(f"Predicted Character: {predicted_character}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 402ms/step
Predicted Character: ں


In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
directory_path = "/content/dataset/json"
data, labels = load_data_from_directory(directory_path)

    # Preprocess the data
sequences, labels_encoded, label_encoder = preprocess_data(data, labels)
X_train, X_test, y_train, y_test = train_test_split(sequences, labels_encoded, test_size=0.2, random_state=42)
# Function to evaluate the model
def evaluate_model(model, X_test, y_test, label_encoder):
    # Predict the labels
      y_pred = model.predict(X_test)
      y_pred_classes = np.argmax(y_pred, axis=1)  # Get class indices from predictions
      y_true = np.argmax(y_test, axis=1)  # Convert one-hot encoded y_test to class indices

    # Calculate performance metrics
      accuracy = accuracy_score(y_true, y_pred_classes)
      precision = precision_score(y_true, y_pred_classes, average='weighted')
      recall = recall_score(y_true, y_pred_classes, average='weighted')
      f1 = f1_score(y_true, y_pred_classes, average='weighted')

    # Create a DataFrame for easy visualization
      performance_metrics = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Score': [accuracy, precision, recall, f1]
      })

      print(performance_metrics)

evaluate_model(model, X_test, y_test, label_encoder)


566/566 ━━━━━━━━━━━━━━━━━━━━ 21s 36ms/step
      Metric     Score
0   Accuracy  0.092692
1  Precision  0.099765
2     Recall  0.092692
3   F1 Score  0.060326


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
